In [129]:
import pyomo.environ as pyo
from pyomo.util.model_size import build_model_size_report

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [130]:
date_parser = pd.to_datetime
combined_market_data = pd.read_csv('../build_dataset/combined_market_data.csv', index_col=0, parse_dates=True, date_parser=date_parser)
combined_market_data

/var/folders/28/cmh8kslx4f9gr_zckx5qp_sw0000gn/T/ipykernel_72593/1949149456.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  combined_market_data = pd.read_csv('../build_dataset/combined_market_data.csv', index_col=0, parse_dates=True, date_parser=date_parser)


,IPindex_netztransparenz_15min_pE,aFRRpos_SMARD_15min_E,aFRRneg_SMARD_15min_E,aFRRpos_SMARD_15min_pE,aFRRneg_SMARD_15min_pE,aFRRpos_SMARD_15min_P,aFRRneg_SMARD_15min_P,aFRRpos_SMARD_15min_pP,aFRRneg_SMARD_15min_pP,FCR_regelleistung_1D_pP,FCR_regelleistung_4h_pP,FCR_SMARD_15min_pP,DADEATLU_ENTSOE_60min_pE,DADELU_ENTSOE_60min_pE,DADELU_ENTSOE_15min_pE
2015-01-01 00:00:00,NaN,2.0,101.0,58.63,59.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:15:00,NaN,5.0,34.0,61.69,30.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:30:00,NaN,30.0,7.0,71.03,4.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:45:00,NaN,15.0,9.0,66.68,7.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 01:00:00,NaN,77.0,1.0,76.44,-2.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-10 22:45:00,115.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-10 23:00:00,114.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-10 23:15:00,97.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-10 23:30:00,99.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
DA_IP = combined_market_data[['IPindex_netztransparenz_15min_pE','DADELU_ENTSOE_60min_pE']]
DA_IP_lim = DA_IP[DA_IP['DADELU_ENTSOE_60min_pE'].first_valid_index():DA_IP['DADELU_ENTSOE_60min_pE'].last_valid_index()][DA_IP['IPindex_netztransparenz_15min_pE'].first_valid_index():DA_IP['IPindex_netztransparenz_15min_pE'].last_valid_index()]
#ax2 = DA_IP_lim['IPindex_netztransparenz_15min_pE'].plot(ylim=(-100,1000))
# ax1 = DA_IP_lim['IPindex_netztransparenz_15min_pE'].plot()
# ax2 = DA_IP_lim['DADELU_ENTSOE_60min_pE'].dropna().plot(ylim=(-100,3000))
# lgd = ax1.legend(['IP Index', '60min day-ahead spot'], bbox_to_anchor=(1,0.5))
# ax1.set_ylabel('Price (EUR/MWh)')
# ax1.set_xlabel('Date')

In [152]:
#DA_IP_lim['IPindex_netztransparenz_15min_pE']
DA = DA_IP_lim['DADELU_ENTSOE_60min_pE']
IP = DA_IP_lim['IPindex_netztransparenz_15min_pE']
DA = DA.fillna(DA.shift(24, freq='h')).asfreq('1h')
IP = IP.fillna(IP.shift(96, freq='15min'))  # 96 * 15min = 24 hours


DA = DA[DA.index < DA.last_valid_index()] #- pd.Timedelta('1min')]
IP = IP[IP.index < DA.last_valid_index() + pd.Timedelta('60min')]

# Check both indexes are continuous (should be 1 for first index)
print(sum(DA.index.diff() != pd.Timedelta('1h')))
print(sum(IP.index.diff() != pd.Timedelta('15min')))

# Check all indexes match when IP is resampled to 1h (should be 0)
print((DA.index != IP.asfreq('1h').index).sum()) 


1
1
0
